# Part 7 - Model Comparison
Let's load and compare the models we trained in the previous notebook. 

In [ ]:
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
df = pd.read_csv('./data/rew_van_jan12_clean_engineered.csv')
features = [feature for feature in df.columns if feature != 'price']
X = df[features]
y = df['price']

In [ ]:
def load_models():
    with open('./models/multiple_linear.pkl', 'rb') as f:
        linear_model = pickle.load(f)
    with open('./models/xgb.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('./models/random_forest.pkl', 'rb') as f:
        rf_model = pickle.load(f)
    with open('./models/mlp.pkl', 'rb') as f:
        mlp_model = pickle.load(f)
    models = [linear_model, xgb_model, rf_model, mlp_model]
    return models
models = load_models()

In [ ]:
# try brand new data
# https://www.rew.ca/properties/R2230074/1897-w-37th-avenue-vancouver-bc?search_type=property_browse&search_id=vancouver-bc
actual_price = '$5,688,000'
sqft = 3790
bed = 4
bath = 4
sub_area = "Quilchena"
area = "Vancouver West"
property_type = "House"
strata_type = "Freehold NonStrata"
new_data = {'sqft': sqft,
            'bed': bed,
            'bath': bath,
            'sub_area_{}'.format(sub_area): 1,
            'area_{}'.format(area): 1,
            'property_type_{}'.format(property_type): 1,
            'strata_type_{}'.format(strata_type): 1
           }
new_df = pd.get_dummies(pd.DataFrame(data=[new_data], columns=X.columns).fillna(0))

for model in models:
    predicted_price = model.predict(new_df)
    print("method: {}".format(model.__class__))
    print("predicted price: ${}M".format(predicted_price[0]/1e6))
    print("actual price: {}".format(actual_price))

In [ ]:
# try brand new data
actual_price = '$3M'
sqft = 1776
bed = 3
bath = 2
sub_area = "South Granville"
area = "Vancouver West"
property_type = "Apt/Condo"
strata_type = "Freehold Strata"
new_data = {'sqft': sqft,
            'bed': bed,
            'bath': bath,
            'sub_area_{}'.format(sub_area): 1,
            'area_{}'.format(area): 1,
            'property_type_{}'.format(property_type): 1,
            'strata_type_{}'.format(strata_type): 1
           }
new_df = pd.get_dummies(pd.DataFrame(data=[new_data], columns=X.columns).fillna(0))
for model in models:
    predicted_price = model.predict(new_df)
    print("method: {}".format(model.__class__))
    print("predicted price: ${}M".format(predicted_price[0]/1e6))
    print("actual price: {}".format(actual_price))